In [2]:
import deap
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from deap import base, creator, tools, algorithms
import random
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp


2025-06-20 23:41:21.953905: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-20 23:41:21.957889: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-20 23:41:21.965706: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750477281.978695     329 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750477281.982365     329 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-20 23:41:21.997027: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Parametros del problema

In [ ]:
L = 1
g = 9.81
m = 0.25


# $Individuo = [W_1, B_1, W_2, B_2]$

In [ ]:
def WeightsPlacer(individual, tuner):
    '''
    Toma los pesos, que se encuentran en el individuo
    y los convierte en forma tensorial para poder
    ser colocados en el modelo
    '''
    best_config = tuner.get_best_hyperparameters(num_trials=1)[0]
    model = tuner.hypermodel.build(best_config) 
    idx = 0
    tensors=[]
    for layer in model.layers[1:]: #Excluye la capa Normalizadora, que ya se entrenó
        for weight_arr in layer.get_weights():

            size = weight_arr.size

            shape = weight_arr.shape
            tensor = individual[idx:idx+size]
            
            tensor = np.array(tensor)
            tensor = np.reshape(tensor, shape)

            idx += size

            tensors.append(tensor)

    model.set_weights(tensors)

    return model

def Fitness_Function(individual, train_dataset, batch_size, step):
    #Create the model
     #CHANGE THIS BUILD USING HYPERPARMS
    #Set the weights

    model = WeightsPlacer(individual)

    #for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
    (x_batch_train, y_batch_train) = train_dataset
    
    current_batch_X = x_batch_train[step*batch_size:(step+1)*batch_size]
    current_batch_Y = y_batch_train[step*batch_size:(step+1)*batch_size]

    mae_per_batch = np.abs(model.predict(x_batch_train) - y_batch_train)

    Average_training_torque_error = np.mean(mae_per_batch) 

    return Average_training_torque
             

In [ ]:
def Training_Loop(population, toolbox, mu, lambda_, cxpb, mutpb, ngen,train_dataset, batch_size, step,
                   stats=None, halloffame=None, verbose=__debug__ ):
    r"""This is the :math:`(\mu + \lambda)` evolutionary algorithm.

    :param population: A list of individuals.
    :param toolbox: A :class:`~deap.base.Toolbox` that contains the evolution
                    operators.
    :param mu: The number of individuals to select for the next generation.
    :param lambda\_: The number of children to produce at each generation.
    :param cxpb: The probability that an offspring is produced by crossover.
    :param mutpb: The probability that an offspring is produced by mutation.
    :param ngen: The number of generation.
    :param stats: A :class:`~deap.tools.Statistics` object that is updated
                  inplace, optional.
    :param halloffame: A :class:`~deap.tools.HallOfFame` object that will
                       contain the best individuals, optional.
    :param verbose: Whether or not to log the statistics.
    :returns: The final population
    :returns: A class:`~deap.tools.Logbook` with the statistics of the
              evolution.

    The algorithm takes in a population and evolves it in place using the
    :func:`varOr` function. It returns the optimized population and a
    :class:`~deap.tools.Logbook` with the statistics of the evolution. The
    logbook will contain the generation number, the number of evaluations for
    each generation and the statistics if a :class:`~deap.tools.Statistics` is
    given as argument. The *cxpb* and *mutpb* arguments are passed to the
    :func:`varOr` function. The pseudocode goes as follow ::

        evaluate(population)
        for g in range(ngen):
            offspring = varOr(population, toolbox, lambda_, cxpb, mutpb)
            evaluate(offspring)
            population = select(population + offspring, mu)

    First, the individuals having an invalid fitness are evaluated. Second,
    the evolutionary loop begins by producing *lambda_* offspring from the
    population, the offspring are generated by the :func:`varOr` function. The
    offspring are then evaluated and the next generation population is
    selected from both the offspring **and** the population. Finally, when
    *ngen* generations are done, the algorithm returns a tuple with the final
    population and a :class:`~deap.tools.Logbook` of the evolution.

    This function expects :meth:`toolbox.mate`, :meth:`toolbox.mutate`,
    :meth:`toolbox.select` and :meth:`toolbox.evaluate` aliases to be
    registered in the toolbox. This algorithm uses the :func:`varOr`
    variation.
    """
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = []
    curr_step = 0
    for ind in invalid_ind:
        fitnesses.append(Fitness_Function(ind, train_dataset, batch_size, curr_step))

    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is not None:
        halloffame.update(population)

    record = stats.compile(population) if stats is not None else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Begin the generational process
    for gen in range(1, ngen + 1):
        # Vary the population
        offspring = varOr(population, toolbox, lambda_, cxpb, mutpb)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        #fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        fitnesses = []
        for ind in invalid_ind:
            fitnesses.append(Fitness_Function(ind, train_dataset, batch_size, gen))
            
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the hall of fame with the generated individuals
        if halloffame is not None:
            halloffame.update(offspring)

        # Select the next generation population
        population[:] = toolbox.select(population + offspring, mu)

        # Update the statistics with the new population
        record = stats.compile(population) if stats is not None else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook

In [ ]:

# Pendulum dynamics (no friction)
def pendulum_ode(t, y, g=9.81, L=1.0):
    theta, omega = y
    dtheta_dt = omega
    domega_dt = (g / L) * np.sin(theta)
    return [dtheta_dt, domega_dt]

# Initial conditions
theta0 = np.radians(0)  # initial angle in radians
omega0 = 0.0            # initial angular velocity
y0 = [theta0, omega0]

# Time span and step


def calculate_next_timestep():
    t_span = (0, 1)
    t_eval = np.linspace(t_span[0], t_span[1], 1000)
    # Solve using SciPy RK45
    sol = solve_ivp(
        pendulum_ode,
        t_span,
        y0,
        t_eval=t_eval,
        args=(g, L),        # optional: pass g and L
        method="RK45",           # can also try "RK23", "DOP853", "BDF", etc.
        rtol=1e-9,
        atol=1e-9
    )
    return sol

# Extract results
theta = sol.y[0]
omega = sol.y[1]
accel = (9.81 / 1.0) * np.sin(theta)

# Optional: convert to linear quantities
L = 1.0
x = L * np.sin(theta)
v = L * omega * np.cos(theta)
a = L * accel * np.cos(theta) - L * omega**2 * np.sin(theta)

# Plotting
plt.plot(sol.t, theta, label="Angle (rad)")
plt.plot(sol.t, omega, label="Angular velocity (rad/s)")
plt.plot(sol.t, accel, label="Angular acceleration (rad/s²)")
plt.legend()
plt.xlabel("Time (s)")
plt.title("Inverted Pendulum (No Friction)")
plt.grid(True)
plt.show()
